In [1]:
%matplotlib widget

In [2]:
import numpy as np
import tables
import flammkuchen as fl
from pathlib import Path
from tqdm import tqdm
from fimpylab.utils.suite2p_conv import export_suite2p
import tifffile as tiff
import matplotlib.pyplot as plt

from fimpylab.core.lightsheet_experiment import LightsheetExperiment

import json
from scipy.optimize import quadratic_assignment
from lotr.pca import qap_sorting_and_phase

import statsmodels.api as sm
from scipy.cluster.hierarchy import dendrogram, cut_tree, set_link_color_palette
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, to_tree, set_link_color_palette
from sklearn.cluster import AgglomerativeClustering

In [3]:
def cluster_id_search(tree):
    nodes_list = []
    if tree.is_leaf():
        nodes_list.append(tree.get_id())
    else:
        nodes_list += cluster_id_search(tree.get_left())
        nodes_list += cluster_id_search(tree.get_right())
        
    return nodes_list

def find_trunc_dendro_clusters(linkage_mat, dendro):
    tree, branches = to_tree(linkage_mat, rd =True)
    ids = np.empty(linkage_mat.shape[0]+1, dtype=int)
    
    for i, clust in enumerate(dendro["leaves"]):
        ids[cluster_id_search(branches[clust])] = i
        
    return ids

In [3]:
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v29_29p1s\ls\1sec")
files = list(master_path.glob("*_f*"))
print(len(files))

11


In [4]:
fish_dir = files[2]
suite2p_data = fl.load(fish_dir / 'data_from_suite2p_cells.h5')
roi_coords = suite2p_data['coords']

'''
traces = suite2p_data['traces']
norm_traces = np.copy(traces)
norm_traces = norm_traces.T
sd = np.nanstd(norm_traces, 0)
mean = np.nanmean(norm_traces, 0)
norm_traces = norm_traces - mean
norm_traces = norm_traces / sd
'''

norm_traces = fl.load(fish_dir / 'filtered_traces.h5')['detr']
norm_traces = norm_traces.T

num_rois, len_rec = np.shape(norm_traces)
print(num_rois, len_rec)

t_start = 50
norm_traces = norm_traces[:, t_start:]


32742 3600


In [5]:
corr_thresh = -0.4
corrmat = np.corrcoef(norm_traces)
ng_corr_ind = np.unique(np.where(corrmat < corr_thresh)[0])

corrmat_neg = corrmat[ng_corr_ind]
norm_traces_neg = norm_traces[ng_corr_ind]

num_ng = np.shape(ng_corr_ind)[0]
min_val = np.nanmin(corrmat, axis=0)



In [6]:
n_clust = 12
fig_tmp = plt.figure()
linked = linkage(norm_traces_neg, method='ward')
dend = dendrogram(linked)
cluster = AgglomerativeClustering(n_clusters=n_clust, affinity='euclidean', linkage='ward')
her_clustering = cluster.fit_predict(norm_traces_neg)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
panel_dendro = dendrogram(linked,
                          #orientation='left',
                          distance_sort='descending',
                          show_leaf_counts=False,
                          no_labels=True,
                          above_threshold_color='#%02x%02x%02x' % (
                          120, 120, 120))

In [8]:
cluster_ids = dend["leaves"]
labels = find_trunc_dendro_clusters(linked, dend)

NameError: name 'find_trunc_dendro_clusters' is not defined

In [ ]:
labels = cluster.labels_

In [108]:
exp = LightsheetExperiment(fish_dir)
tail = exp.behavior_log['tail_sum']
roi_coords_neg = roi_coords[ng_corr_ind]

fig_all, ax = plt.subplots(3, 2, figsize=(10,8), gridspec_kw={'width_ratios':[1, 3], 'height_ratios': [1, 6, 6]})

ax[0,0].axis('off')
ax[1,0].scatter(roi_coords[:, 1], roi_coords[:, 2], c=min_val, cmap='hot', s=3, vmin=-1, vmax=0)
ax[1,0].axis('off')
ax[1,0].set_title('Minimal correlation')

ax[2,0].axis('off')
ax[2,0].set_title('Minimal correlated neurons')

ax[0,1].plot(tail, c='k')
ax[0,1].set_xlim((t_start / 2 * 400, len(tail)))
ax[0,1].axis('off')


ax[1,1].imshow(norm_traces_neg[panel_dendro["leaves"], :], cmap='coolwarm', vmin=-2, vmax=2, aspect='auto')
ax[1,1].axis('off')


col_list =  ["lightblue", "lightcoral", "orange", "springgreen", "deepskyblue", "mediumpurple","gold", "cyan", "crimson",
              "deeppink", "lawngreen", "darkviolet", "Darkgreen", "blue", "brown", "dodgerblue", "hotpink", "OliveDrab", "gray", "seagreen"][0:n_clust]
for i in range(n_clust):
    ax[2,0].scatter(roi_coords_neg[labels == i, 1], roi_coords_neg[labels == i, 2], c=col_list[i], s=3)
    ax[2,1].plot(np.nanmean(norm_traces_neg[labels == i, :], 0) + i * 5, label=i, color=col_list[i])
ax[2,1].axis('off')


file_name = 'ROI_maps_anticorr_hrc_220324.jpg'
fig_all.savefig(fish_dir / file_name, dpi=300)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
exp = LightsheetExperiment(fish_dir)
tail = exp.behavior_log['tail_sum']
roi_coords_neg = roi_coords[ng_corr_ind]

fig_all2, ax2 = plt.subplots(3, 2, figsize=(10,8), gridspec_kw={'width_ratios':[1, 3], 'height_ratios': [1, 6, 6]})

ax2[0,0].axis('off')
ax2[1,0].scatter(roi_coords[:, 1], roi_coords[:, 2], c=min_val, cmap='hot', s=3, vmin=-1, vmax=0)
ax2[1,0].axis('off')
ax2[1,0].set_title('Minimal correlation')

ax2[2,0].axis('off')
ax[2,0].set_title('Minimal correlated neurons')

ax2[0,1].plot(tail, c='k')
ax2[0,1].set_xlim((t_start / 2 * 400, len(tail)))
ax2[0,1].axis('off')


norm_traces_0 = norm_traces_neg[labels == 0, :]
norm_traces_5 = norm_traces_neg[labels == 5, :]
norm_traces_05 = np.concatenate((norm_traces_0, norm_traces_5), axis=0)
ax2[1,1].imshow(norm_traces_05, cmap='coolwarm', vmin=-2, vmax=2, aspect='auto')
ax2[1,1].axis('off')


col_list =  ["lightblue", "lightcoral", "orange", "springgreen", "deepskyblue", "mediumpurple","gold", "cyan", "crimson",
              "deeppink", "lawngreen", "darkviolet", "Darkgreen", "blue", "brown", "dodgerblue", "hotpink", "OliveDrab", "gray", "seagreen"][0:n_clust]
for i in [0,5]:
    ax2[2,0].scatter(roi_coords_neg[labels == i, 1], roi_coords_neg[labels == i, 2], c=col_list[i], s=3)
    ax2[2,1].plot(np.nanmean(norm_traces_neg[labels == i, :], 0) + i, label=i, color=col_list[i])
ax2[2,1].axis('off')
ax2[2,0].set_ylim(0, 700)
ax2[2,0].set_xlim(0, 700)

file_name = 'ROI_maps_anticorr_hrc_220324_part.jpg'
fig_all2.savefig(fish_dir / file_name, dpi=300)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
data = {'selected': norm_traces_05}
fl.save(fish_dir / 'selected_from_hrc.h5', data)

In [99]:
corrmat = np.corrcoef(norm_traces_05)
norm_traces05_2 = norm_traces_05[np.unique(np.where(corrmat < -0.6)[0])]
perm, com_phase = qap_sorting_and_phase(norm_traces_05.T)
sorted_traces = norm_traces_05[perm]
sorted_corrmat = np.corrcoef(sorted_traces)


In [100]:
fig_all_sort, ax_sort = plt.subplots(2, 1, figsize=(6,4), gridspec_kw={'height_ratios': [1, 6]})

ax_sort[0].plot(tail, color='k')
ax_sort[0].set_aspect('auto')
ax_sort[0].axis('off')
ax_sort[0].set_title('Tail sum')
ax_sort[0].set_xlim((t_start / 2 * 400, len(tail)))

ax_sort[1].imshow(sorted_traces, cmap="coolwarm", vmin=-2, vmax=2, aspect='auto')
ax_sort[1].axis('off')
ax_sort[1].set_title('Negatively correlated traces')

file_name = fish_dir / "tail_sorted_traces_06.jpg"
fig_all_sort.savefig(str(file_name), dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
fig5, ax5 = plt.subplots(3, 4, figsize=(12, 12), sharex=True, sharey=True)


for i in range(n_clust): 
    r = i // 4
    c = np.mod(i, 4)
    print(r,c)
    ax5[r, c].scatter(roi_coords_neg[labels == i, 1], roi_coords_neg[labels == i, 2], c=col_list[i], s=3)
    ax5[r,c].axis('off')


plt.show()
file_name = 'clusters_rois_hrc_' + '_k' + str(n_clust) + '.jpg'
fig5.savefig(str(fish_dir/file_name), dpi=300)

C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
